In [9]:
import torch
import torch.nn as nn

import os

from pathlib import Path
from utils.loader import get_save_path
from utils.var_dim import squeezeToNumpy
import numpy as np
import matplotlib.pyplot as plt
from imageio import imread
from tensorboardX import SummaryWriter
from utils.utils import getWriterPath
from datasets.patches_dataset import PatchesDataset
from utils.logging import *
import yaml

In [10]:
hpatches_datasets = PatchesDataset()

In [11]:
hpatches_datasets

In [12]:
from utils.loader import dataLoader, modelLoader, pretrainedLoader

In [13]:
output_dir='/Users/kweonminseong/Documents/git/Matching-Algorithm/outputs/hpatches'
with open('/Users/kweonminseong/Documents/git/Matching-Algorithm/config/magicpoint_repeatability_heatmap.yaml') as f:
    config = yaml.safe_load(f)

In [16]:
from models.model_wrap import PointTracker

# basic settings
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
logging.info("train on device: %s", device)
with open(os.path.join(output_dir, "config.yml"), "w") as f:
    yaml.dump(config, f, default_flow_style=False)
save_path = get_save_path(output_dir)
save_output = save_path / "../predictions"
os.makedirs(save_output, exist_ok=True)

## parameters
outputMatches = True
subpixel = config["model"]["subpixel"]["enable"]
patch_size = config["model"]["subpixel"]["patch_size"]

# data loading
from utils.loader import dataLoader_test as dataLoader
task = config["data"]["dataset"]
data = dataLoader(config, dataset=task)
test_set, test_loader = data["test_set"], data["test_loader"]
from utils.print_tool import datasize
datasize(test_loader, config, tag="test")

# model loading
from utils.loader import get_module
Val_model_heatmap = get_module("", config["front_end_model"])
## load pretrained
val_agent = Val_model_heatmap(config["model"], device=device)
val_agent.loadModel()

## tracker
tracker = PointTracker(max_length=2, nn_thresh=val_agent.nn_thresh)

2023-08-14 19:26:26 minseong-MacBookPro.local root[40653] INFO train on device: cpu
2023-08-14 19:26:26 minseong-MacBookPro.local root[40653] INFO => will save everything to /Users/kweonminseong/Documents/git/Matching-Algorithm/outputs/hpatches/checkpoints
2023-08-14 19:26:26 minseong-MacBookPro.local root[40653] INFO workers_test: 1
2023-08-14 19:26:26 minseong-MacBookPro.local root[40653] INFO == test split size 580 in 580 batches
2023-08-14 19:26:26 minseong-MacBookPro.local root[40653] INFO => creating model: SuperPointNet_gauss2
2023-08-14 19:26:26 minseong-MacBookPro.local root[40653] INFO successfully load pretrained model from: logs/superpoint_coco_heat2_0/checkpoints/superPointNet_170000_checkpoint.pth.tar


In [22]:
from tqdm import tqdm


###### check!!!
count = 0
for i, sample in tqdm(enumerate(test_loader)):
    img_0, img_1 = sample["image"], sample["warped_image"]

    # first image, no matches
    # img = img_0
    def get_pts_desc_from_agent(val_agent, img, device="cpu"):
        """
        pts: list [numpy (3, N)]
        desc: list [numpy (256, N)]
        """
        heatmap_batch = val_agent.run(
            img.to(device)
        )  # heatmap: numpy [batch, 1, H, W]
        # heatmap to pts
        pts = val_agent.heatmap_to_pts()
        # print("pts: ", pts)
        if subpixel:
            pts = val_agent.soft_argmax_points(pts, patch_size=patch_size)
        # heatmap, pts to desc
        desc_sparse = val_agent.desc_to_sparseDesc()
        # print("pts[0]: ", pts[0].shape, ", desc_sparse[0]: ", desc_sparse[0].shape)
        # print("pts[0]: ", pts[0].shape)
        outs = {"pts": pts[0], "desc": desc_sparse[0]}
        return outs

    def transpose_np_dict(outs):
        for entry in list(outs):
            outs[entry] = outs[entry].transpose()

    outs = get_pts_desc_from_agent(val_agent, img_0, device=device)
    pts, desc = outs["pts"], outs["desc"]  # pts: np [3, N]

    if outputMatches == True:
        tracker.update(pts, desc)

    # save keypoints
    pred = {"image": squeezeToNumpy(img_0)}
    pred.update({"prob": pts.transpose(), "desc": desc.transpose()})

    # second image, output matches
    outs = get_pts_desc_from_agent(val_agent, img_1, device=device)
    pts, desc = outs["pts"], outs["desc"]

    if outputMatches == True:
        tracker.update(pts, desc)

    pred.update({"warped_image": squeezeToNumpy(img_1)})
    # print("total points: ", pts.shape)
    pred.update(
        {
            "warped_prob": pts.transpose(),
            "warped_desc": desc.transpose(),
            "homography": squeezeToNumpy(sample["homography"]),
        }
    )

    if outputMatches == True:
        matches = tracker.get_matches()
        print("matches: ", matches.transpose().shape)
        pred.update({"matches": matches.transpose()})
    print("pts: ", pts.shape, ", desc: ", desc.shape)

    # clean last descriptor
    tracker.clear_desc()

    filename = str(count)
    path = Path(save_output, "{}.npz".format(filename))
    np.savez_compressed(path, **pred)
    # print("save: ", path)
    count += 1
print("output pairs: ", count)

0it [00:00, ?it/s]/Users/kweonminseong/.pyenv/versions/matching-algorithm/lib/python3.10/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/TensorShape.cpp:3484.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
1it [00:00,  1.72it/s]

heatmap:  (244, 324)
heatmap:  (244, 324)
matches:  (256, 4)
pts:  (3, 407) , desc:  (256, 407)


2it [00:00,  2.59it/s]

heatmap:  (244, 324)
heatmap:  (244, 324)
matches:  (248, 4)
pts:  (3, 393) , desc:  (256, 393)


3it [00:01,  3.10it/s]

...

heatmap:  (244, 324)
heatmap:  (244, 324)
matches:  (282, 4)
pts:  (3, 368) , desc:  (256, 368)


580it [03:00,  3.52it/s]

heatmap:  (244, 324)
heatmap:  (244, 324)
matches:  (77, 4)
pts:  (3, 336) , desc:  (256, 336)


580it [03:05,  3.13it/s]

output pairs:  580
